<a href="https://colab.research.google.com/github/gacevedob/Proyectos/blob/main/4D_Timeliner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install specklepy --upgrade

In [ ]:
import requests
import pandas as pd

# Configuración general
HOST = "https://speckle.xyz/graphql"
TOKEN = "d59135bf7dada88ba2d3db54695f81606a7a6e86b0"

HEADERS = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}

In [ ]:
# Función para realizar consultas a Speckle
def make_speckle_query(query, variables):
    try:
        response = requests.post(HOST, json={"query": query, "variables": variables}, headers=HEADERS)
        response.raise_for_status()  # Lanza una excepción si la solicitud falla
        return response.json()
    except requests.RequestException as e:
        print(f"Error en la solicitud: {e}")
        return {}


In [ ]:
# Consulta y variables
QUERY = """
query commit($projectId: String!, $objectId: String!, $limit: Int!) {
  project(id: $projectId) {
    object(id: $objectId) {
      children(limit: $limit) {
        objects {
          data
        }
      }
    }
  }
}
"""
VARIABLES = {
    "projectId": "bc02abaa2c",
    "objectId": "b421eb5d599db197fe8189589a0cad00",
    "limit": 10000
}


In [ ]:
# Ejecutar la consulta
response = make_speckle_query(QUERY, VARIABLES)

In [ ]:
# Extraer los datos relevantes
data = response.get("data", {}).get("project", {}).get("object", {}).get("children", {}).get("objects", [])

# Crear una lista para almacenar los resultados procesados
results = []

In [ ]:
from datetime import datetime

# Función para convertir fechas al formato DD-MM-YYYY
def convertir_fecha(fecha):
    try:
        # Detectar y procesar la fecha solo si es válida
        if fecha:
            return datetime.strptime(fecha, "%Y-%m-%d").strftime("%d-%m-%Y")
    except ValueError:
        return fecha  # Si no es una fecha válida, se retorna como está
    return None

# Extraer y procesar los datos relevantes
def extract_results(data):
    results = []
    for obj in data:
        obj_data = obj.get("data", {})
        properties = obj_data.get("properties", {})
        elemento = properties.get("Elemento", {})
        componente = properties.get("Componente", {})
        timeliner = properties.get("TimeLiner", {})

        # Filtrar resultados con categoría no nula
        categoria = componente.get("Categoría")
        if categoria:
            results.append({
                "id": obj_data.get("id"),
                "Categoría": categoria,
                "Nivel": elemento.get("Capa"),
                "Tarea": timeliner.get("Contenido en tarea:1"),
                "Inicio Planeado": convertir_fecha(timeliner.get("Contenido en inicio de tarea (Planeada):1")),
                "Fin Planeado": convertir_fecha(timeliner.get("Contenido en fin de tarea (Planeada):1")),
                "Inicio Real": convertir_fecha(timeliner.get("Contenido en inicio de tarea (Real):1")),
                "Fin Real": convertir_fecha(timeliner.get("Contenido en fin de tarea (Real):1")),
            })
    return results

# Crear resultados
results = extract_results(data)

# Verificar los resultados
print(results[:5])  # Muestra los primeros 5 resultados procesados


[{'id': '0040953dd205001c3b9036ebdbd57be8', 'Categoría': 'Cimentación estructural', 'Nivel': '01 - Entry Level', 'Tarea': 'Encepado sector 1', 'Inicio Planeado': '09/28/2023 18:45:36', 'Fin Planeado': '10/20/2023 18:45:36', 'Inicio Real': '10/10/2023 18:45:36', 'Fin Real': '10/16/2023 18:45:36'}, {'id': '004dd62d2510b59825e5c214036d67aa', 'Categoría': 'Armazón estructural', 'Nivel': '02 - Floor', 'Tarea': 'Vigas sector 2', 'Inicio Planeado': '01/18/2024 16:43:12', 'Fin Planeado': '02/14/2024 10:01:48', 'Inicio Real': '01/30/2024 16:43:12', 'Fin Real': '02/13/2024 16:43:12'}, {'id': '00553356d130789ad2515b87ce51db63', 'Categoría': 'Cimentación estructural', 'Nivel': '01 - Entry Level', 'Tarea': 'Pilotes sector 1', 'Inicio Planeado': '09/11/2023 09:00:00', 'Fin Planeado': '09/28/2023 18:45:36', 'Inicio Real': '09/25/2023 09:00:00', 'Fin Real': '10/11/2023 19:00:00'}, {'id': '0097716adec52a81cfaad860c95430c1', 'Categoría': 'Armazón estructural', 'Nivel': '03 - Floor', 'Tarea': 'Vigas sect

In [ ]:
# Crear DataFrame
df = pd.DataFrame(results)

# Agregar columnas adicionales
projectId = "bc02abaa2c"
objectId = "b421eb5d599db197fe8189589a0cad00"
df["ModelURL"] = f"https://app.speckle.systems/streams/{projectId}"
df["VersionObjectId"] = objectId

# Verificar el DataFrame
print(df.head())


                                 id                Categoría  \
0  0040953dd205001c3b9036ebdbd57be8  Cimentación estructural   
1  004dd62d2510b59825e5c214036d67aa      Armazón estructural   
2  00553356d130789ad2515b87ce51db63  Cimentación estructural   
3  0097716adec52a81cfaad860c95430c1      Armazón estructural   
4  00cb18cea3a78331cc821a929a1f8b52  Cimentación estructural   

              Nivel              Tarea      Inicio Planeado  \
0  01 - Entry Level  Encepado sector 1  09/28/2023 18:45:36   
1        02 - Floor     Vigas sector 2  01/18/2024 16:43:12   
2  01 - Entry Level   Pilotes sector 1  09/11/2023 09:00:00   
3        03 - Floor     Vigas sector 2  04/04/2024 10:01:48   
4  01 - Entry Level   Pilotes sector 4  09/11/2023 09:00:00   

          Fin Planeado          Inicio Real             Fin Real  \
0  10/20/2023 18:45:36  10/10/2023 18:45:36  10/16/2023 18:45:36   
1  02/14/2024 10:01:48  01/30/2024 16:43:12  02/13/2024 16:43:12   
2  09/28/2023 18:45:36  09/25/20

In [ ]:
# Ordenar el DataFrame por "Inicio Planeado"
df = df.sort_values("Inicio Planeado")

# Mostrar el DataFrame final
df


,id,Categoría,Nivel,Tarea,Inicio Planeado,Fin Planeado,Inicio Real,Fin Real,ModelURL,VersionObjectId
586,49b40864aaea3a1fbe61faca346ff49b,Armazón estructural,03 - Floor,Vigas sector 1,01/10/2024 17:34:48,01/26/2024 17:34:48,None,None,https://app.speckle.systems/streams/bc02abaa2c,b421eb5d599db197fe8189589a0cad00
1637,d0b3bd2afbe7ecde7391c2bcd131cfc0,Armazón estructural,03 - Floor,Vigas sector 1,01/10/2024 17:34:48,01/26/2024 17:34:48,None,None,https://app.speckle.systems/streams/bc02abaa2c,b421eb5d599db197fe8189589a0cad00
1635,d035eeb27bd9e6cbebcab9e344cbd040,Armazón estructural,03 - Floor,Vigas sector 1,01/10/2024 17:34:48,01/26/2024 17:34:48,None,None,https://app.speckle.systems/streams/bc02abaa2c,b421eb5d599db197fe8189589a0cad00
151,1358e43ba985702fea123da6d8442c34,Armazón estructural,03 - Floor,Vigas sector 1,01/10/2024 17:34:48,01/26/2024 17:34:48,None,None,https://app.speckle.systems/streams/bc02abaa2c,b421eb5d599db197fe8189589a0cad00
1521,c26c969ed9fa2bf70af3ad18493a4d69,Armazón estructural,03 - Floor,Vigas sector 1,01/10/2024 17:34:48,01/26/2024 17:34:48,None,None,https://app.speckle.systems/streams/bc02abaa2c,b421eb5d599db197fe8189589a0cad00
...,...,...,...,...,...,...,...,...,...,...
806,6810e4c5a35bdf477c30ff9bd60e3d77,Pilares estructurales,02 - Floor,Pilares sector 1,12/26/2023 17:34:48,01/10/2024 17:34:48,01/02/2024 17:34:48,01/17/2024 17:34:48,https://app.speckle.systems/streams/bc02abaa2c,b421eb5d599db197fe8189589a0cad00
517,41e3f8308564166bd22ed76ac0b3372c,Pilares estructurales,02 - Floor,Pilares sector 1,12/26/2023 17:34:48,01/10/2024 17:34:48,01/02/2024 17:34:48,01/17/2024 17:34:48,https://app.speckle.systems/streams/bc02abaa2c,b421eb5d599db197fe8189589a0cad00
1160,92429cb09a0a831e1215362beb3440c0,Pilares estructurales,02 - Floor,Pilares sector 1,12/26/2023 17:34:48,01/10/2024 17:34:48,01/02/2024 17:34:48,01/17/2024 17:34:48,https://app.speckle.systems/streams/bc02abaa2c,b421eb5d599db197fe8189589a0cad00
781,6507cdcf0d7780c7dae773bb2542ce6b,Pilares estructurales,02 - Floor,Pilares sector 1,12/26/2023 17:34:48,01/10/2024 17:34:48,01/02/2024 17:34:48,01/17/2024 17:34:48,https://app.speckle.systems/streams/bc02abaa2c,b421eb5d599db197fe8189589a0cad00


In [ ]:
# --- Google Sheets Integration ---
from google.colab import auth
from googleapiclient.discovery import build
import time


In [ ]:
from google.colab import auth
from googleapiclient.discovery import build

# Autenticar el acceso a la API de Google Sheets
auth.authenticate_user()

# Crear el servicio de Google Sheets API
service = build('sheets', 'v4')

# ID del libro de Google Sheets 'Timeliner_4D_Data' (reemplaza este ID con el real de tu libro)
spreadsheet_id = '1muBCs590gVkqbWtO1xTXP-rsMrPaJoj82IlcT0WLqVg'  # Reemplázalo con el ID de tu libro 'Timeliner_4D_Data'

# Convertir el DataFrame a una lista de listas (formato que requiere Google Sheets)
values = df.values.tolist()

# Agregar los encabezados (nombres de las columnas) como la primera fila
encabezados = df.columns.tolist()
values.insert(0, encabezados)  # Inserta los encabezados como la primera fila

# Nombre de la hoja será el objectId
nombre_hoja = f"{objectId}"

# Obtener las hojas existentes del libro
spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
sheets = spreadsheet.get('sheets', [])

# Verificar si la hoja con el objectId ya existe
hoja_existente = False
for sheet in sheets:
    if sheet['properties']['title'] == nombre_hoja:
        hoja_existente = True
        break

if hoja_existente:
    # Si la hoja ya existe, actualizamos los datos en lugar de crearla
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=f"{nombre_hoja}!A1",  # Especifica la celda donde comenzar a escribir
        valueInputOption="RAW",
        body={"values": values}
    ).execute()

    print(f"Datos actualizados en la hoja '{nombre_hoja}' en el libro '{spreadsheet_id}'.")
else:
    # Si la hoja no existe, la creamos
    add_sheet_request = {
        "requests": [
            {
                "addSheet": {
                    "properties": {
                        "title": nombre_hoja  # Título de la nueva hoja
                    }
                }
            }
        ]
    }

    # Crear la nueva hoja
    service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body=add_sheet_request
    ).execute()

    # Escribir los datos en la nueva hoja
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=f"{nombre_hoja}!A1",  # Especifica la celda donde comenzar a escribir
        valueInputOption="RAW",
        body={"values": values}
    ).execute()

    print(f"Hoja '{nombre_hoja}' creada y datos escritos en el libro '{spreadsheet_id}'.")


Datos actualizados en la hoja 'b421eb5d599db197fe8189589a0cad00' en el libro '1muBCs590gVkqbWtO1xTXP-rsMrPaJoj82IlcT0WLqVg'.
